In [2]:
import cv2
import time
import torch
import numpy as np
from ultralytics import YOLO
from collections import Counter
import os


In [3]:
torch.device("cuda" if torch.cuda.is_available() else "cpu")
yolo = YOLO("yolov8n.pt")


In [12]:
!yolo task=detect mode=predict model=yolov8n.pt source=image.png project=./runs conf=0.01

Ultralytics 8.3.7 🚀 Python-3.10.15 torch-2.4.1 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)
YOLOv8n summary (fused): 168 layers, 3,151,904 parameters, 0 gradients, 8.7 GFLOPs

image 1/1 c:\Users\DHO_d\OneDrive\Escritorio\football_ieee\football-analysis\image.png: 384x640 91 persons, 1 sports ball, 1 skateboard, 1 chair, 46.9ms
Speed: 0.0ms preprocess, 46.9ms inference, 152.0ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs\predict
💡 Learn more at https://docs.ultralytics.com/modes/predict


In [4]:
cv2.imread("image.png")
result = yolo("image.png")
result


image 1/1 c:\Users\DHO_d\OneDrive\Escritorio\football_ieee\football-analysis\image.png: 384x640 8 persons, 60.0ms
Speed: 9.9ms preprocess, 60.0ms inference, 179.4ms postprocess per image at shape (1, 3, 384, 640)


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted p

In [4]:

class VideoProcessor:
    def __init__(self, modelo='yolov8n.pt', source='camera', video_path=None, ruta_carpeta=None, save = False, n = 1,depurar = False):
        self.yolo_model = YOLO(modelo)
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print(f"Using device: {self.device}")
        self.yolo_model.to(self.device)
        
        self.source = source
        self.n = n
        self.depurar = depurar
        if source == 'camera':
            self.cap = cv2.VideoCapture(0)  # 0 para la cámara por defecto
        elif source == 'video' and video_path:
            self.cap = cv2.VideoCapture(video_path)
        else:
            raise ValueError("Invalid source or missing video path")
        
        if not self.cap.isOpened():
            raise RuntimeError(f"Error: Could not open {'camera' if source == 'camera' else 'video file'}.")
        else:
            print(f"{'Camera' if source == 'camera' else 'Video file'} opened successfully.")
        
        # Precalcular colores para las detecciones
        self.colors = np.random.randint(0, 255, size=(80, 3), dtype='uint8')
        
        self.ruta_carpeta = ruta_carpeta
        if ruta_carpeta:
            os.makedirs(self.ruta_carpeta, exist_ok=True)
        self.save = save
    def procesar(self, num_frames=None, frame_skip=0):
        frames_leidos = 0
        frames_procesados = 0
        total_time = 0

        while self.cap.isOpened():
            if num_frames is not None and frames_procesados >= num_frames:
                break

            for _ in range(frame_skip + 1):
                ret, frame = self.cap.read()
                if not ret:
                    print("Fin del video o error de lectura.")
                    return
                frames_leidos += 1

            start_time = time.time()

            # Factor de escala
            self.n = 1
            original_height, original_width = frame.shape[:2]
            frame = cv2.resize(frame, (int(original_width * self.n), int(original_height * self.n)))

            # Detectar solo personas con batch processing
            results = self.yolo_model(frame, stream=True,classes = [0,32] ,conf=0.1)  # 0 es el índice para 'person'

            # Procesar resultados
            for result in results:
                detections = result.boxes
                print(f"Frame {frames_leidos} detecciones: {len(detections)}")
                frame_with_detections = self.process_detections(frame, detections)
                
                # Mostrar el frame con las detecciones
                cv2.imshow('Detecciones', frame_with_detections)
                if self.save == True:
                    self.guardar_frame(frame_with_detections,frames_leidos)

            end_time = time.time()
            elapsed_time = end_time - start_time
            total_time += elapsed_time

            print(f"Tiempo de procesamiento del frame {frames_leidos}: {elapsed_time:.4f} segundos")

            frames_procesados += 1

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        avg_time = total_time / frames_procesados if frames_procesados > 0 else 0
        print(f"Frames procesados: {frames_procesados}")
        print(f"Tiempo promedio de procesamiento por frame: {avg_time:.4f} segundos")
        print(f"FPS promedio: {1/avg_time:.2f}")

        self.cap.release()
        cv2.destroyAllWindows()
        print("Captura de video liberada.")
        
    def modify_frame(self, frame):
        hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        
        # Máscara para colores azules
        mask_blue = cv2.inRange(hsv_frame, np.array([75, 50, 50]), np.array([135, 255, 255]))
        
        # Suavizar y limpiar la máscara (operaciones morfológicas)
        #kernel = np.ones((5, 5), np.uint8)
        #mask_blue = cv2.GaussianBlur(mask_blue, (5, 5), 0)
        #mask_blue = cv2.morphologyEx(mask_blue, cv2.MORPH_CLOSE, kernel)
        #mask_blue = cv2.morphologyEx(mask_blue, cv2.MORPH_OPEN, kernel)
        
        # Reducir saturación en áreas azules
        hsv_frame[:, :, 1] = np.where(mask_blue > 0, 0, hsv_frame[:, :, 1])
        
        # Ajustar HSV perfil rojo amarillo
        hsv_frame[:, :, 0] = np.clip(hsv_frame[:, :, 0], 37, 179)
        hsv_frame[:, :, 2] = 255  # Maximizar el valor (brillo)
        #Ajustar HSV perfil blancos
        #hsv_frame[:, :, 0] = np.clip(hsv_frame[:, :, 0], 29, 132)  # Asegúrate de que H no exceda los límites
        #hsv_frame[:, :, 2] = 255  # Maximizar el valor (brillo)

        # ajustar vivos 

        H_low, H_high = 0, 179
        S_low, S_high = 0, 255
        V_low, V_high = 255, 255

        # Aplicar los límites a los canales H, S, y V
        #hsv_frame[:, :, 0] = np.clip(hsv_frame[:, :, 0], H_low, H_high)  # Canal H
        #hsv_frame[:, :, 1] = np.clip(hsv_frame[:, :, 1], S_low, S_high)  # Canal S
        #hsv_frame[:, :, 2] = np.clip(hsv_frame[:, :, 2], V_low, V_high)  # Canal V
        
        return cv2.cvtColor(hsv_frame, cv2.COLOR_HSV2BGR)

    def process_detections(self, frame, detections):
        modified_frame = self.modify_frame(frame)
        player_colors = []
        
        for detection in detections:
            if len(detection.xyxy) >= 1:
                x1, y1, x2, y2 = map(int, detection.xyxy[0][:4])
                mid_x = (x1 + x2) // 2
                mid_y = (y1 + y2) // 2
                half_height = (y2 - y1) // 2
                half_width = (x2 - x1) // 2
                centered_x1 = max(x1, mid_x - half_width // 2)
                centered_x2 = min(x2, mid_x + half_width // 2)
                centered_y1 = max(y1, mid_y - half_height // 2)
                centered_y2 = min(y2, mid_y + half_height // 2)

                player_img = modified_frame[centered_y1:centered_y2, centered_x1:centered_x2]
                avg_color_hsv = cv2.cvtColor(np.uint8([[player_img.mean(axis=(0,1))]]), cv2.COLOR_BGR2HSV)[0][0]
                player_colors.append(tuple(avg_color_hsv.astype(int)))
        
        team_colors = self.get_team_colors(player_colors)
        if self.depurar == False:
            frame_with_detections = self.draw_detections(frame.copy(), detections, team_colors)
        else:
            frame_with_detections = self.draw_detections(modified_frame, detections, team_colors)
        # Mostrar el frame con las detecciones
        cv2.imshow('Detecciones', frame_with_detections)
        
        return frame_with_detections

    def get_team_colors(self, player_colors):
        color_counts = Counter(player_colors)
        unique_colors = list(color_counts.keys())
        
        if len(unique_colors) < 2:
            return None
        
        distances = np.linalg.norm(np.array(unique_colors)[:, np.newaxis] - np.array(unique_colors), axis=2)
        i, j = np.unravel_index(distances.argmax(), distances.shape)
        
        return tuple(sorted([unique_colors[i], unique_colors[j]]))

    def draw_detections(self, frame, detections, team_colors):
        for detection in detections:
            if len(detection.xyxy) >= 1:
                x1, y1, x2, y2 = map(int, detection.xyxy[0][:4])
                mid_x = (x1 + x2) // 2
                mid_y = (y1 + y2) // 2
                half_height = (y2 - y1) // 2
                half_width = (x2 - x1) // 2
                centered_x1 = max(x1, mid_x - half_width // 2)
                centered_x2 = min(x2, mid_x + half_width // 2)
                centered_y1 = max(y1, mid_y - half_height // 2)
                centered_y2 = min(y2, mid_y + half_height // 2)
                
                player_img = frame[centered_y1:centered_y2, centered_x1:centered_x2]
                avg_color_hsv = cv2.cvtColor(np.uint8([[player_img.mean(axis=(0,1))]]), cv2.COLOR_BGR2HSV)[0][0]
                
                team = 1 if team_colors and np.linalg.norm(avg_color_hsv - team_colors[0]) < np.linalg.norm(avg_color_hsv - team_colors[1]) else 2
                
                cv2.rectangle(frame, (centered_x1, centered_y1), (centered_x2, centered_y2), cv2.cvtColor(np.uint8([[avg_color_hsv]]), cv2.COLOR_HSV2BGR)[0][0].tolist(), 1)
                cv2.circle(frame, ((x1+x2)//2, (y1+y2)//2), 3, cv2.cvtColor(np.uint8([[avg_color_hsv]]), cv2.COLOR_HSV2BGR)[0][0].tolist(), -1)
                cv2.putText(frame, f'Equipo {team}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)
        
        return frame


    def guardar_frame(self, frame, frame_index):
            if self.ruta_carpeta is None:
                print("No se ha especificado una carpeta para guardar los frames.")
                return
            try:
                os.makedirs(self.ruta_carpeta, exist_ok=True)
                output_frame_filename = f"frame_clasificado_{frame_index}.png"
                saved_path = os.path.join(self.ruta_carpeta, output_frame_filename)
                cv2.imwrite(saved_path, frame)
                if os.path.exists(saved_path):
                    print(f'Imagen del frame guardada como {output_frame_filename}')
                else:
                    print(f'Error al guardar la imagen del frame {frame_index}.')
            except Exception as e:
                print(f"Error al guardar el frame {frame_index}: {str(e)}")

In [13]:
def modify_frame(frame):
    frame = cv2.imread(frame)
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # Lower all colors by reducing the saturation and value
    hsv_frame[:, :, 1] = hsv_frame[:, :, 1] * 0.5  # Reduce saturation by 50%
    hsv_frame[:, :, 2] = hsv_frame[:, :, 2] * 0.5  # Reduce value (brightness) by 50%
    
    # Intensify yellow
    mask_yellow = cv2.inRange(hsv_frame, np.array([20, 50, 50]), np.array([30, 255, 255]))
    hsv_frame[:, :, 1] = np.where(mask_yellow > 0, hsv_frame[:, :, 1] * 2, hsv_frame[:, :, 1])  # Increase saturation for yellow
    hsv_frame[:, :, 2] = np.where(mask_yellow > 0, hsv_frame[:, :, 2] * 2, hsv_frame[:, :, 2])  # Increase value for yellow
    
    return cv2.cvtColor(hsv_frame, cv2.COLOR_HSV2BGR)

frame = modify_frame("image.png")
# save frame
cv2.imwrite("x/image.png", frame)


True

In [ ]:
processor = VideoProcessor(modelo='yolov8n.pt', source='camera', n = 1, save = True, ruta_carpeta='pruebavivo5',depurar=True)
processor.procesar()

In [9]:
processor = VideoProcessor(modelo='yolov8n.pt', source='video', video_path='s.mp4',ruta_carpeta = 'vivo2',save = True,n = 1,depurar=False)
processor.procesar(num_frames=10, frame_skip=100)

Using device: cuda
Video file opened successfully.

0: 640x384 14 persons, 10.0ms
Frame 101 detecciones: 14
Imagen del frame guardada como frame_clasificado_101.png
Speed: 10.2ms preprocess, 10.0ms inference, 4.5ms postprocess per image at shape (1, 3, 640, 384)
Tiempo de procesamiento del frame 101: 0.3098 segundos

0: 640x384 7 persons, 13.5ms
Frame 202 detecciones: 7
Imagen del frame guardada como frame_clasificado_202.png
Speed: 0.0ms preprocess, 13.5ms inference, 2.5ms postprocess per image at shape (1, 3, 640, 384)
Tiempo de procesamiento del frame 202: 0.0850 segundos

0: 640x384 4 persons, 10.1ms
Frame 303 detecciones: 4
Imagen del frame guardada como frame_clasificado_303.png
Speed: 0.0ms preprocess, 10.1ms inference, 3.9ms postprocess per image at shape (1, 3, 640, 384)
Tiempo de procesamiento del frame 303: 0.0801 segundos

0: 640x384 2 persons, 1 sports ball, 11.0ms
Frame 404 detecciones: 3
Imagen del frame guardada como frame_clasificado_404.png
Speed: 0.0ms preprocess, 11